In [1]:
import os
import json
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess

In [35]:
# Load the API key and set the model name
file_path = "../keys.json"
with open(file_path, "r", encoding="utf-8") as file:
    key_data = json.load(file)

os.environ["OPENAI_API_KEY"] = key_data["openai_api_key"]
model_name = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.1, model_name=model_name)


In [23]:
# Load the dataset JSON file
data_path = "../data/tqa_train_val_test/train/tqa_v1_train.json"
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# For a quick check, print the first record
print(data[0])

{'adjunctTopics': {'Apply Concepts': {'content': {'figures': [], 'mediaLinks': [], 'text': '5. A glacier is melting. What are all of the scientists you can think of who might be involved in studying this glacier? What would each of them do? '}, 'orderID': 't_12'}, 'Introduction': {'content': {'figures': [{'caption': 'FIGURE 1.10 Earth as seen from Apollo 17.', 'imagePath': 'textbook_images/earth_science_and_its_branches_20010.png'}], 'mediaLinks': [], 'text': 'Earth Science is the study of all aspects of our planet Earth. Earth Science is not just about the molten lava, icy mountain peaks, steep canyons and towering waterfalls of the continents. Earth Science includes the atmosphere and oceans. The field also looks out into the solar system, galaxy, and universe. Earth scientists seek to understand the beautiful planet on which we depend (Figure 1.10). Different branches of Earth Science study one particular part of Earth. Since all of the branches are connected, specialists work toget

In [4]:
def extract_textbook_text(entry):
    """
    Extract textbook text from a TQA dataset entry.
    Retrieves text from the 'adjunctTopics' and 'topics' sections.
    """
    texts = []
    
    # Extract text from adjunctTopics
    adjunct_topics = entry.get("adjunctTopics", {})
    for topic in adjunct_topics.values():
        content = topic.get("content", {})
        text = content.get("text", "")
        if text:
            texts.append(text.strip())
            
    # Extract text from topics
    topics = entry.get("topics", {})
    for topic in topics.values():
        content = topic.get("content", {})
        text = content.get("text", "")
        if text:
            texts.append(text.strip())
    
    return texts

In [5]:
import os

def extract_textbook_text_with_image_explainations(entry):
    """
    Extract textbook text from a TQA dataset entry.
    Retrieves text from:
      - 'lessonName'
      - 'adjunctTopics' and their content text and figures (with LLava explanations)
      - 'topics' and their content text and figures (with LLava explanations)
    
    For figures, this function assumes that:
      - Figures with image paths containing "textbook_images" have their LLava explanations 
        stored in the "textbook_images_llava_captions" folder.
      - Figures with image paths containing "teaching_images" have their LLava explanations 
        stored in the "teaching_images_llava_captions" folder.
    
    Explanation files are assumed to be named as the image file name (without its extension)
    plus a ".txt" extension, and are located under the base directory:
    "../data/tqa_train_val_test/train/"
    """
    texts = []
    
    # Include lesson title
    lesson_name = entry.get("lessonName", "")
    if lesson_name:
        texts.append(f"Lesson: {lesson_name.strip()}")
    
    # Base directory for explanation files
    base_dir = "../data/tqa_train_val_test/train/"
    
    def extract_content_and_explanations(section):
        section_texts = []
        content = section.get("content", {})
        
        # Append the text content if available
        text = content.get("text", "")
        if text:
            section_texts.append(text.strip())
        
        # Process figures for captions and explanations
        figures = content.get("figures", [])
        for fig in figures:
            caption = fig.get("caption", "").strip()
            image_path = fig.get("imagePath", "")
            if image_path:
                # Determine the appropriate explanation folder based on the image path
                if "textbook_images" in image_path:
                    explanation_folder = "textbook_images_llava_captions"
                elif "teaching_images" in image_path:
                    explanation_folder = "teaching_images_llava_captions"
                else:
                    explanation_folder = None
                
                if explanation_folder:
                    file_name = os.path.basename(image_path)
                    # Remove file extension and add .txt
                    base_name = os.path.splitext(file_name)[0]
                    explanation_file = os.path.join(base_dir, explanation_folder, base_name + ".txt")
                    # print("Looking for explanation file:", explanation_file)
                    if os.path.exists(explanation_file):
                        with open(explanation_file, 'r') as f:
                            explanation = f.read().strip()
                        if explanation:
                            # Combine caption and explanation in a single string
                            combined_text = ""
                            if caption:
                                combined_text += f"Image Caption: {caption}. "
                            combined_text += f"Image Explanation: {explanation}"
                            section_texts.append(combined_text)
        return section_texts

    # Extract text and explanations from adjunctTopics
    adjunct_topics = entry.get("adjunctTopics", {})
    for topic in adjunct_topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    # Extract text and explanations from topics
    topics = entry.get("topics", {})
    for topic in topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    return texts


In [6]:
# Combine the extracted text from the first record
extracted_texts = extract_textbook_text_with_image_explainations(data[0])
combined_text = "\n\n".join(extracted_texts)
print("Combined Text:\n")
print(combined_text)

Combined Text:

Lesson: earth science and its branches

5. A glacier is melting. What are all of the scientists you can think of who might be involved in studying this glacier? What would each of them do?

Earth Science is the study of all aspects of our planet Earth. Earth Science is not just about the molten lava, icy mountain peaks, steep canyons and towering waterfalls of the continents. Earth Science includes the atmosphere and oceans. The field also looks out into the solar system, galaxy, and universe. Earth scientists seek to understand the beautiful planet on which we depend (Figure 1.10). Different branches of Earth Science study one particular part of Earth. Since all of the branches are connected, specialists work together to answer complicated questions. Lets look at some important branches of Earth Science.

Image Caption: FIGURE 1.10 Earth as seen from Apollo 17.. Image Explanation: The image shows a close-up view of the Earth from space. The Earth is surrounded by a bla

In [7]:
# ########################
# # 1. Generate Mindmap(v1)  #
# ########################
# def generate_mindmap(text, llm, max_tokens=500):
#     """
#     Generate a mind map in Mermaid syntax for the given text using ChatOpenAI (or similar LLM).
#     The mind map should capture the main topics and their subtopics in a clear hierarchical structure.
#     IMPORTANT: We instruct the model not to use a generic 'root' node, but rather a relevant main topic
#     as the central node for better readability.
#     """
#     prompt = (
#         "Generate a mind map in Mermaid syntax for the following textbook text. "
#         "The mind map should capture the main topics and their subtopics clearly in a hierarchical structure. "
#         "Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics "
#         "as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. "
#         "Also, do NOT include any theme directives such as %%{init: {\"theme\": \"default\"}}%%, any parentheses or any extraneous formatting."
#         "The mind map should be self-explanatory so that by reading it, a user can understand the key content and structure "
#         "of the text document. Keep the Mermaid syntax minimal so it renders correctly, and be concise.\n\n"
#         f"{text}\n\n"
#         "Mermaid Mind Map:"
#     )
    
#     result = llm.invoke(prompt, max_tokens=max_tokens)
#     mindmap = result.content.strip()
#     return mindmap

In [38]:
########################
# 1. Generate Mindmap(v2)  #
########################
def generate_mindmap(text, llm, max_tokens=500):
    """
    Generate a mind map in Mermaid syntax for the given text using ChatOpenAI (or similar LLM).
    The mind map should capture the main topics and their subtopics in a clear hierarchical structure.
    IMPORTANT: We instruct the model not to use a generic 'root' node, but rather a relevant main topic
    as the central node for better readability.
    """
    prompt = (
        "Generate a mind map in Mermaid syntax for the following textbook text. "
        "Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics "
        "as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. "
        "Also, do NOT include any theme directives such as %%{init: {\"theme\": \"default\"}}%%, any parentheses, (, ) or any extraneous formatting. strictly no ( or )"
        "The mind map should be self-explanatory so that by reading it, a user can understand the key content of the text document."
        "Keep the Mermaid syntax minimal so it renders correctly.\n\n"
        f"{text}\n\n"
        "Mermaid Mind Map:"
    )

    result = llm.invoke(prompt, max_tokens=max_tokens)
    mindmap = result.content.strip()
    return mindmap

In [9]:
############################
# 2. Clean Mermaid Code    #
############################
import re

def clean_mermaid_code(code):
    # Split into lines and remove any markdown fences or theme directives
    lines = code.splitlines()
    cleaned_lines = []
    for line in lines:
        if line.strip().startswith("```"):
            continue
        if "%%{init:" in line:
            continue
        cleaned_lines.append(line)
    return "\n".join(cleaned_lines)


In [10]:
from IPython.display import SVG, display
import subprocess
import os

def render_mermaid(mindmap_code, output_filename='mermaid_output.svg', theme='neutral', background_color=None, css_file=None):
    """
    Render a Mermaid diagram as an SVG file using the Mermaid CLI (mmdc).
    
    Parameters:
    - mindmap_code: Mermaid syntax code to render.
    - output_filename: Name of the output SVG file.
    - theme: Mermaid theme to use (e.g., 'forest', 'dark', 'neutral').
    - background_color: Optional background color (e.g., "#ffffff").
    - css_file: Optional path to a custom CSS file for additional styling.
    
    The function writes the Mermaid code to a temporary file, invokes the Mermaid CLI,
    displays the generated SVG inline (Jupyter/IPython), and cleans up temporary files.
    """
    # Write Mermaid code to a temporary file
    temp_file = "temp_mermaid.mmd"
    with open(temp_file, "w", encoding="utf-8") as f:
        f.write(mindmap_code)
    
    # Build the command for Mermaid CLI
    cmd = ["mmdc", "-i", temp_file, "-o", output_filename, "-t", theme]
    
    if background_color:
        cmd.extend(["-b", background_color])
    
    if css_file:
        cmd.extend(["-c", css_file])
    
    subprocess.run(cmd, check=True)
    
    # Display the generated SVG inline (Jupyter/IPython)
    # display(SVG(output_filename))
    
    os.remove(temp_file)


In [11]:
# Generate the Mermaid mind map for the combined textbook text
mindmap_code = generate_mindmap(combined_text, llm)
print("\nGenerated Mind Map (Mermaid format):\n")
print(mindmap_code)


Generated Mind Map (Mermaid format):

```mermaid
mindmap
  Earth Science
    Definition
      Study of all aspects of Earth
      Includes atmosphere and oceans
      Explores solar system and universe
    Branches
      Geology
        Study of rocks and minerals
        Processes of Earth
        Specializations
          Mineralogy
          Volcanology
          Seismology
          Paleontology
          Planetary geology
      Oceanography
        Study of oceans
        Water movement and chemistry
        Specializations
          Physical oceanography
          Marine geology
          Chemical oceanography
          Marine biology
      Meteorology
        Study of atmosphere
        Weather patterns and climate
        Importance of forecasting
      Environmental Science
        Human impact on environment
        Ecological studies
      Astronomy
        Study of solar system and universe
        Tools for exploration
        Questions about existence and resources
    I

In [12]:

# 2) Clean code to remove markdown fences
cleaned_code = clean_mermaid_code(mindmap_code)
print("Cleaned Mermaid code:")
print(cleaned_code)

# 3) Render with the desired theme (e.g., 'forest', 'dark', 'neutral')
render_mermaid(cleaned_code, 'mermaid_output.svg', theme='forest')

Cleaned Mermaid code:
mindmap
  Earth Science
    Definition
      Study of all aspects of Earth
      Includes atmosphere and oceans
      Explores solar system and universe
    Branches
      Geology
        Study of rocks and minerals
        Processes of Earth
        Specializations
          Mineralogy
          Volcanology
          Seismology
          Paleontology
          Planetary geology
      Oceanography
        Study of oceans
        Water movement and chemistry
        Specializations
          Physical oceanography
          Marine geology
          Chemical oceanography
          Marine biology
      Meteorology
        Study of atmosphere
        Weather patterns and climate
        Importance of forecasting
      Environmental Science
        Human impact on environment
        Ecological studies
      Astronomy
        Study of solar system and universe
        Tools for exploration
        Questions about existence and resources
    Importance
      Understandin

## Run on folder

In [36]:
import os

data_path = "../data/tqa_train_val_test/train/tqa_v1_train.json"
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
data = data

In [37]:
# print(data)

In [39]:
import os
from tqdm import tqdm  # progress bar library

# Determine the dataset directory based on the data_path
dataset_folder = os.path.dirname(data_path)

# Create separate output folders in the dataset directory for mindmap text and SVG
mindmap_text_folder = os.path.join(dataset_folder, "mindmap_text_v2")
svg_folder = os.path.join(dataset_folder, "svg_v2")

os.makedirs(mindmap_text_folder, exist_ok=True)
os.makedirs(svg_folder, exist_ok=True)

# File to log failed entries
failed_log_path = os.path.join(dataset_folder, "failed_entries.txt")

# Clear the failed log file if it exists
with open(failed_log_path, "w", encoding="utf-8") as log_file:
    log_file.write("Failed Entries Log:\n")

# Iterate over the full dataset to generate and render mind maps for each entry with a progress bar
for i, entry in tqdm(enumerate(data), total=len(data), desc="Processing entries"):
    try:
        # Use a dataset identifier if available, otherwise fallback to the index
        identifier = entry.get("globalID", f"entry_{i}")
        
        # Extract the textbook text with image explanations from the current entry
        extracted_texts = extract_textbook_text_with_image_explainations(entry)
        
        # Combine the texts into a single string separated by newlines
        combined_text = "\n\n".join(extracted_texts)
        
        # If there is no text to process, skip this entry
        if not combined_text.strip():
            continue
        
        # Generate the Mermaid mind map using the combined text
        mindmap_code = generate_mindmap(combined_text, llm)
        
        # Clean the generated Mermaid code
        cleaned_code = clean_mermaid_code(mindmap_code)
        
        # Define unique output filenames using the identifier
        txt_filename = os.path.join(mindmap_text_folder, f"mermaid_code_{identifier}.txt")
        svg_filename = os.path.join(svg_folder, f"mermaid_output_{identifier}.svg")
        
        # Save the cleaned Mermaid code to a text file
        with open(txt_filename, "w", encoding="utf-8") as txt_file:
            txt_file.write(cleaned_code)
        
        # Render and display the Mermaid diagram using the 'forest' theme, saving to the SVG folder
        render_mermaid(cleaned_code, svg_filename, theme='forest')
        
    except Exception as e:
        error_message = f"Error processing entry {i} (ID: {identifier}): {e}\n"
        print(error_message)
        with open(failed_log_path, "a", encoding="utf-8") as log_file:
            log_file.write(error_message)
        continue


Processing entries:   0%|                                                                                                                                                                                                                                    | 0/666 [00:00<?, ?it/s]

Generating single mermaid chart


Processing entries:   0%|▎                                                                                                                                                                                                                           | 1/666 [00:05<59:55,  5.41s/it]

Generating single mermaid chart


Processing entries:   0%|▋                                                                                                                                                                                                                           | 2/666 [00:10<56:08,  5.07s/it]

...

Processing entries: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 664/666 [48:34<00:07,  3.96s/it]

Generating single mermaid chart


Processing entries: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 665/666 [48:37<00:03,  3.73s/it]

Generating single mermaid chart


Processing entries: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666/666 [48:40<00:00,  4.38s/it]


In [40]:
import os
from tqdm import tqdm  # progress bar library

# Determine the dataset directory based on the data_path
dataset_folder = os.path.dirname(data_path)

# Create separate output folders in the dataset directory for mindmap text and SVG
mindmap_text_folder = os.path.join(dataset_folder, "mindmap_text_v2")
svg_folder = os.path.join(dataset_folder, "svg_v2")

os.makedirs(mindmap_text_folder, exist_ok=True)
os.makedirs(svg_folder, exist_ok=True)

# File to log failed entries
failed_log_path = os.path.join(dataset_folder, "failed_entries.txt")

# Clear the failed log file if it exists
with open(failed_log_path, "w", encoding="utf-8") as log_file:
    log_file.write("Failed Entries Log:\n")

# Iterate over the full dataset to generate and render mind maps for each entry with a progress bar
for i, entry in tqdm(enumerate(data), total=len(data), desc="Processing entries"):
    try:
        # Use a dataset identifier if available, otherwise fallback to the index
        identifier = entry.get("globalID", f"entry_{i}")
        
        # Define unique output filenames using the identifier
        txt_filename = os.path.join(mindmap_text_folder, f"mermaid_code_{identifier}.txt")
        svg_filename = os.path.join(svg_folder, f"mermaid_output_{identifier}.svg")
        
        # Skip this entry if the SVG file already exists
        if os.path.exists(svg_filename):
            continue
        
        # Extract the textbook text with image explanations from the current entry
        extracted_texts = extract_textbook_text_with_image_explainations(entry)
        
        # Combine the texts into a single string separated by newlines
        combined_text = "\n\n".join(extracted_texts)
        
        # If there is no text to process, skip this entry
        if not combined_text.strip():
            continue
        
        # Generate the Mermaid mind map using the combined text
        mindmap_code = generate_mindmap(combined_text, llm)
        
        # Clean the generated Mermaid code
        cleaned_code = clean_mermaid_code(mindmap_code)
        
        # Save the cleaned Mermaid code to a text file
        with open(txt_filename, "w", encoding="utf-8") as txt_file:
            txt_file.write(cleaned_code)
        
        # Render and display the Mermaid diagram using the 'forest' theme, saving to the SVG folder
        render_mermaid(cleaned_code, svg_filename, theme='forest')
        
    except Exception as e:
        error_message = f"Error processing entry {i} (ID: {identifier}): {e}\n"
        print(error_message)
        with open(failed_log_path, "a", encoding="utf-8") as log_file:
            log_file.write(error_message)
        continue


Processing entries: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666/666 [00:00<00:00, 16213.02it/s]
